In [9]:
'''
这里需要用到数据增强，在transforms模块里实现
进一步在transforms里完成数据预处理
DataLoader模块直接读取batch数据
'''
import os
import torch
import matplotlib.pyplot as plt
from torchvision import transforms, datasets, models
'''
transforms:常用的图像预处理方法
datasets:常用数据集的dataset实现，MNIST，CIFAR-10，Image-net等
models:常用的预训练模型，AlexNet，VGG，ResNet，GoogleNet等
'''
from torch import nn



### 网络模块设置：

- 加载预训练模型，torchvision中有很多经典网络架构，调用起来十分方便，并且可以用训练好的权重参数来继续训练，也就是所谓的迁移学习
- 需要注意的是别人训练好的任务跟我们自己的任务可不是完全一样，需要把最后的head层改一改，一般也就是最后的全连接层，改成自己的任务
- 训练时可以全部重头训练，也可以只训练最后咱们任务的层，因为前几层都是做特征提取的，本质任务目标是一致的

### 网络模型保存与测试
- 模型保存的时候可以带有选择性，例如在验证集中如果当前效果好则保存
- 读取模型进行实际测试

In [10]:
# 数据路径定义 ———— 这里的数据集采用同一类数据放在相同的文件夹里，用文件夹的标号来作为label
data_dir = './flower_data/'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [11]:
# 数据预处理
data_transforms = {
    'train':
        transforms.Compose([
            transforms.Resize([96, 96]),  # 图片大小裁剪为相同
            '''
            数据不够，要高效利用现有数据，做数据增强，让数据获得多样性''',
            transforms.RandomRotation(45),  # 随机旋转 -45° ~ 45°
            transforms.CenterCrop(64),  # 从中心开始裁剪，最后送入模型的大小为64*64
            transforms.RandomHorizontalFlip(p=0.5),  # 随机水平翻转图像，概率值取0.5
            transforms.RandomVerticalFlip(p=0.5),   # 随机垂直翻转，概率值取0.5
            transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),#参数1为亮度，参数2为对比度，参数3为饱和度，参数4为色相
            transforms.RandomGrayscale(p=0.025),    # 概率转换成灰度率，3通道就是R=G=B
            transforms.ToTensor(),  # 数据转换为Tensor结构
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 数据标准化(分别对RGB三个通道)，均值，标准差 (x - μ)/σ
        ]),
    'valid':
        transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}